In [1]:
import pathlib
import textwrap
import os
import google.generativeai as genai
from IPython.display import display, Markdown, HTML
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
import json
import re
import tiktoken


In [2]:
# Load environment variables
load_dotenv(find_dotenv())
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

# Configure the API client with the API key
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-1.5-pro')
validation_model = genai.GenerativeModel('gemini-1.5-pro')  # Third LLM for validation


In [3]:
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def upload_rubric(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content


In [4]:
def format_prompt(rubric_content, essay_text, additional_info):
    return f"""
    Validate the following grading rubric for formatting and appropriateness:

    Grading Rubric:
    {rubric_content}

    Make careful judgement on the subject of the student essay. Is the rubric formatted correctly and appropriate for grading the essay? (yes or no) Please explain why for either case. 
    If no, notify the user that the rubric is not corresponding to the student essay; do not grade the essay, but rather come up with a correct rubric and then the essay.
    If yes, grade the essay with the rubric.

    Student Response:
    {essay_text}

    Additional Information:
    {additional_info}
    
    For Additional Information, please assesses this holistically within the other criteria. Do not add new criteria. 
    

    Sample Output Format:

    ### Grading Rubric and Feedback
    | Criterion | Score | Explanation |
    |-----------|-------|-------------|
    
    Please do not say criterion and the criterion number next to the actual criterion. Always provide an explanation. Furthermore, provide overall feedback.
    
    Calculate the final score by adding up the score for each criterion. If there is a weight, either in a new column or parentheses, please take weighted average.
    """

def format_validation_prompt(feedback, rubric_content, essay_text, additional_info):
    return f"""
    Validate the following grading feedback:

    Grading Feedback:
    {feedback}

    Grading Rubric (ensure this rubric is applied):
    {rubric_content}

    Student Response:
    {essay_text}

    Additional Information:
    {additional_info}
    
    Ensure the feedback follows the instructions set forth in the original prompt and confirm that the original rubric has been applied correctly. The instructions are as follows:
    1. Do not create a new rubric.
    2. Do not add new criteria if the rubric was correct.
    3. Make sure the Additional Information is assessed holistically within the other criteria.
    4. Make sure that the grading scores are logical.
    5. Make sure there are explanations for each criterion.
    6. Ensure that the final score is correctly calculated, considering the specified weights.
    7. Make sure there is overall feedback.

    If the feedback does not follow the instructions, correct it to ensure compliance. Only re-grade if the scores are not logical.
    Provide the corrected grading feedback if the initial feedback was invalid.
    """

def analyze_essay_with_rubric(model, rubric_content, essay_text, additional_info):
    rubric_tokens = model.count_tokens(rubric_content).total_tokens
    essay_tokens = model.count_tokens(essay_text).total_tokens
    additional_info_tokens = model.count_tokens(additional_info).total_tokens

    print(f"Rubric Token Count: {rubric_tokens}")
    print(f"Essay Token Count: {essay_tokens}")
    print(f"Additional Info Token Count: {additional_info_tokens}")
    
    prompt = format_prompt(rubric_content, essay_text, additional_info)
    
    prompt_tokens = model.count_tokens(prompt).total_tokens
    print(f"Grading Prompt Token Count: {prompt_tokens}")
    
    response = model.generate_content(prompt)
    
    response.resolve()
    
    usage_metadata = response.usage_metadata
    print(f"Grading Response Token Count: {usage_metadata.candidates_token_count}")
    print(f"Grading Total Token Count: {usage_metadata.total_token_count}")
    print(" ")
    
    return response.text

def validate_feedback(validation_model, feedback, rubric_content, essay_text, additional_info):
    prompt = format_validation_prompt(feedback, rubric_content, essay_text, additional_info)
    
    response = validation_model.generate_content(prompt)
    
    response.resolve()
    
    usage_metadata = response.usage_metadata
    print(f"Validation Response Token Count: {usage_metadata.candidates_token_count}")
    print(f"Validation Total Token Count: {usage_metadata.total_token_count}")
    print(" ")
    
    return response.text

def get_additional_info():
    preset_options = [
        "Be extra harsh when grading",
        "Add additional encouragement for the student",
        "Focus on grammar and syntax",
        "Consider creativity and originality",
        "Suggest reading materials for the student based on the grading"
    ]
    
    print("Select additional information for grading:")
    for i, option in enumerate(preset_options, 1):
        print(f"{i}. {option}")
    print("6. Enter your own additional information")

    choice = int(input("Enter the number of your choice: "))

    if 1 <= choice <= 5:
        additional_info = preset_options[choice - 1]
    elif choice == 6:
        additional_info = input("Enter your additional information: ")
    else:
        print("Invalid choice, please select a valid option.")
        return get_additional_info()

    return additional_info


In [5]:
'''

def format_validation_prompt(feedback, rubric_content, essay_text, additional_info):
    return f"""
    Validate the following grading feedback:

    Grading Feedback:
    {feedback}

    Grading Rubric (ensure this rubric is applied):
    {rubric_content}

    Student Response:
    {essay_text}

    Additional Information:
    {additional_info}
    
    Ensure the feedback follows the instructions set forth in the original prompt and confirm that the original rubric has been applied correctly. The instructions are as follows:
    1. Do not create a new rubric.
    2. Do not add new criteria if the rubric was correct.
    3. Make sure the Additional Information holistically within the other criteria.
    4. Make sure that the grading scores are logical.
    5. Make sure there are explanations for each criterion.
    6. Make sure the final score is correctly calculated, considering any weights.
    7. Make sure there is overall feedback.

    If the feedback does not follow the instructions, please explain why and correct it to ensure compliance. Only re-grade if the scores are not logical.

    
    Provide the corrected grading feedback if the initial feedback was invalid.

'''

'\n\ndef format_validation_prompt(feedback, rubric_content, essay_text, additional_info):\n    return f"""\n    Validate the following grading feedback:\n\n    Grading Feedback:\n    {feedback}\n\n    Grading Rubric (ensure this rubric is applied):\n    {rubric_content}\n\n    Student Response:\n    {essay_text}\n\n    Additional Information:\n    {additional_info}\n    \n    Ensure the feedback follows the instructions set forth in the original prompt and confirm that the original rubric has been applied correctly. The instructions are as follows:\n    1. Do not create a new rubric.\n    2. Do not add new criteria if the rubric was correct.\n    3. Make sure the Additional Information holistically within the other criteria.\n    4. Make sure that the grading scores are logical.\n    5. Make sure there are explanations for each criterion.\n    6. Make sure the final score is correctly calculated, considering any weights.\n    7. Make sure there is overall feedback.\n\n    If the feedba

In [6]:
def format_suggestions_and_rewrite_prompt(grading_feedback, essay_text):
    return f"""
    Based on the following grading feedback, 
    
    1. Provide suggestions for improvement
    2. Rewrite the essay using the suggested improvements

    Grading Feedback:
    {grading_feedback}

    Essay:
    {essay_text}
    
    Take each improvement provided and identify,
    a. the smallest chunk of the original text where it was applied
    b. the criterion from the rubric 
    c. the reason for the suggested improvement
    
    Provide steps a, b, and c in the following JSON format: 
    [
        {{"improvement": "improvement_1", "criterion_from_rubric": "criterion_from_rubric_1", "reason_for_suggestion": "reason_for_suggestion_1", "original_text": "original_text_1", "revised_text": "revised_text_1"}},
        ...
        {{"improvement": "improvement_n", "criterion_from_rubric": "criterion_from_rubric_n", "reason_for_suggestion": "reason_for_suggestion_n", "original_text": "original_text_n", "revised_text": "original_text_n"}}
    ]
    """

def generate_suggestions_and_rewrite_essay(model, grading_feedback, essay_text):
    feedback_tokens = model.count_tokens(grading_feedback).total_tokens
    essay_tokens = model.count_tokens(essay_text).total_tokens

    print(f"Grading Feedback Token Count: {feedback_tokens}")
    print(f"Essay Token Count: {essay_tokens}")
    
    prompt = format_suggestions_and_rewrite_prompt(grading_feedback, essay_text)
    
    prompt_tokens = model.count_tokens(prompt).total_tokens
    print(f"Suggestions Prompt Token Count: {prompt_tokens}")
    
    response = model.generate_content(prompt)
    
    response.resolve()
    
    usage_metadata = response.usage_metadata
    print(f"Suggestions Response Token Count: {usage_metadata.candidates_token_count}")
    print(f"Suggestions Total Token Count: {usage_metadata.total_token_count}")
    print(" ")
    
    if response and response.text:
        print("Full response from model:\n", response.text)
        return response.text
    else:
        print("Error: No response from model")
        return None


In [7]:
def clean_json_string(json_string):
    json_string = json_string.replace('```json', '').replace('```', '').strip()
    json_string = re.sub(r',\s*([}\]])', r'\1', json_string)
    json_string = re.sub(r'\s*}\s*{', '},{', json_string)
    if json_string.count('[') > json_string.count(']'):
        json_string += ']'
    if json_string.count('{') > json_string.count('}'):
        json_string += '}'
    return json_string

def extract_json_improvements(response):
    if response is None:
        print("Error: No response provided for JSON extraction")
        return {'improvements': [], 'response_without_json': ''}
    try:
        json_match = re.search(r'\[\s*\{.*\}\s*\]', response, re.DOTALL)
        if not json_match:
            raise ValueError("No valid JSON found in response.")
        
        cleaned_json_string = clean_json_string(json_match.group(0))
        print("Cleaned JSON string:", cleaned_json_string)

        improvements = json.loads(cleaned_json_string)
        response_without_json = response.replace(json_match.group(0), '')
        return {'improvements': improvements, 'response_without_json': response_without_json}
    except (ValueError, json.JSONDecodeError) as e:
        print("Error extracting JSON from response:", e)
        print(f"Failed to decode JSON string: {response}")
        return {'improvements': [], 'response_without_json': response}


In [8]:
def display_improvements(improvements, original_text):
    if not improvements:
        print("No improvements to display.")
        return

    css = """
    <style>
    .improvement-tooltip {
        position: relative;
        display: inline-block;
        cursor: pointer;
        background-color: yellow;
        margin-bottom: 10px;
        padding: 5px;
        border: 1px solid black;
        border-radius: 4px;
    }
    .improvement-tooltip .tooltiptext {
        visibility: hidden;
        width: 300px;
        background-color: white;
        color: black;
        text-align: left;
        border: 1px solid #ddd;
        padding: 10px;
        border-radius: 6px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        position: absolute;
        z-index: 1;
        top: 100%;
        left: 50%;
        margin-left: -150px;
    }
    .improvement-tooltip:hover .tooltiptext {
        visibility: visible;
    }
    </style>
    """
    
    html_content = original_text
    
    for improvement in improvements:
        original = improvement.get('original_text', '')
        revised = improvement.get('revised_text', '')
        improvement_text = improvement['improvement']
        reason_for_suggestion = improvement['reason_for_suggestion']
        
        if 'criterion_from_rubric' in improvement:
            criterion = improvement['criterion_from_rubric']
            tooltip_html = f"""
            <div class="improvement-tooltip">
                {revised}
                <span class="tooltiptext">
                    <strong>Criterion:</strong> {criterion}<br>
                    <strong>Improvement:</strong> {improvement_text}<br>
                    <strong>Reason:</strong> {reason_for_suggestion}
                </span>
            </div>
            """
        else:
            tooltip_html = f"""
            <div class="improvement-tooltip">
                {revised}
                <span class="tooltiptext">
                    <strong>Improvement:</strong> {improvement_text}<br>
                    <strong>Reason:</strong> {reason_for_suggestion}
                </span>
            </div>
            """
        
        if original:
            html_content = html_content.replace(original, tooltip_html)
        else:
            html_content = html_content + tooltip_html
    
    display(HTML(css + html_content))


In [9]:
def list_rubrics(rubric_folder):
    return [f for f in os.listdir(rubric_folder) if f.endswith('.md')]

def select_rubric(rubric_folder):
    rubrics = list_rubrics(rubric_folder)
    print("Available rubrics:")
    for idx, rubric in enumerate(rubrics, 1):
        print(f"{idx}. {rubric}")
    
    rubric_index = int(input("Select a rubric by entering the corresponding number: ")) - 1
    selected_rubric = rubrics[rubric_index]
    return os.path.join(rubric_folder, selected_rubric)

current_dir = os.getcwd()
relative_rubric_folder = '../rubrics'
rubric_folder = os.path.join(current_dir, relative_rubric_folder)

if not os.path.exists(rubric_folder):
    print(f"The directory {rubric_folder} does not exist. Creating the directory.")
    os.makedirs(rubric_folder)
    print(f"Please add rubric files in markdown format (.md) to the {rubric_folder} directory and re-run the notebook.")
else:
    rubric_files = list_rubrics(rubric_folder)
    if not rubric_files:
        print(f"No rubric files found in the directory {rubric_folder}. Please add rubric files in markdown format (.md) and re-run the notebook.")
    else:
        rubric_file_path = select_rubric(rubric_folder)
        if rubric_file_path:
            rubric_content = upload_rubric(rubric_file_path)
            display(to_markdown(rubric_content))

additional_info = get_additional_info()


Available rubrics:
1. writing_ela_rubric_11_12_nys.md
2. writing_ela_rubric_9_10_nys.md
3. writing_social_sciences_11_12_nys.md
4. science_rubric_9_12_nys.md
Select a rubric by entering the corresponding number: 4


> # New York State P-12 Science Learning Standards Rubric
> 
> | Criterion                                          | 1                                        | 2                                        | 3                                        | 4                                        | 5                                        |
> |----------------------------------------------------|------------------------------------------|------------------------------------------|------------------------------------------|------------------------------------------|------------------------------------------|
> | **1. Developing and Using Models (20%)**           | No model developed or used.              | Minimal model developed; lacks clarity.  | Basic model developed; some clarity.     | Clear model developed; some synthesis.   | Highly detailed and synthesized model with clear relationships. |
> | **2. Planning and Carrying Out Investigations (25%)** | No investigation planned or carried out. | Minimal planning and execution.          | Basic planning; limited execution.       | Adequate planning and execution.         | Highly thorough and well-executed planning and investigation. |
> | **3. Analyzing and Interpreting Data (20%)**       | No data analyzed or interpreted.         | Minimal analysis; lacks interpretation.  | Basic analysis; some interpretation.     | Clear analysis and interpretation.       | Highly detailed and comprehensive analysis and interpretation. |
> | **4. Engaging in Argument from Evidence (20%)**    | No argument presented.                   | Minimal argument; lacks evidence.        | Basic argument; some evidence.           | Clear argument with evidence.            | Highly detailed and well-supported argument with extensive evidence. |
> | **5. Obtaining, Evaluating, and Communicating Information (15%)** | No information communicated.              | Minimal communication; lacks clarity.    | Basic communication; some clarity.       | Clear communication; adequate evaluation.| Highly detailed and effective communication with thorough evaluation. |


Select additional information for grading:
1. Be extra harsh when grading
2. Add additional encouragement for the student
3. Focus on grammar and syntax
4. Consider creativity and originality
5. Suggest reading materials for the student based on the grading
6. Enter your own additional information
Enter the number of your choice: 1


In [10]:
essay_text =  """
The history of the United States is marked by a continuous struggle for civil rights and equality, shaping the nation's identity and societal structure. From the early days of colonization to the modern era, the quest for civil rights has been a central theme, reflecting the dynamic and often tumultuous nature of the American democratic experiment. This ongoing journey has significantly influenced the development of American society and democracy.
The roots of the civil rights movement can be traced back to the early colonial period, where notions of freedom and equality were paradoxically juxtaposed with the institution of slavery. The Declaration of Independence in 1776, with its proclamation that "all men are created equal," laid a philosophical foundation for civil rights, yet this ideal was not extended to all individuals, particularly African Americans and Native Americans. The contradiction between the ideals of the Declaration and the reality of slavery set the stage for future conflicts and movements aimed at achieving true equality.
The abolitionist movement of the 19th century was one of the earliest organized efforts to challenge slavery and advocate for the civil rights of African Americans. Figures such as Frederick Douglass, Harriet Tubman, and William Lloyd Garrison played pivotal roles in raising awareness and mobilizing support for the abolition of slavery. The publication of antislavery literature, such as Harriet Beecher Stowe's Uncle Tom's Cabin, also contributed to the growing sentiment against slavery in the North.
The culmination of the abolitionist movement was the Civil War (1861-1865), a conflict fundamentally rooted in the issue of slavery. The Union's victory and the subsequent passage of the 13th Amendment in 1865 abolished slavery, marking a significant milestone in the struggle for civil rights. However, the end of slavery did not mean the end of discrimination or the beginning of true equality. The Reconstruction era (1865-1877) saw significant advancements, including the 14th and 15th Amendments, which granted citizenship and voting rights to African Americans. Despite these legal gains, the period was also marked by severe backlash, including the rise of the Ku Klux Klan and the implementation of Jim Crow laws, which enforced racial segregation and disenfranchised Black citizens.
The early 20th century witnessed the emergence of new civil rights organizations and leaders who sought to challenge segregation and discrimination through legal means and grassroots activism. The National Association for the Advancement of Colored People (NAACP), founded in 1909, played a crucial role in challenging discriminatory laws and practices through the court system. One of the most significant victories was the Supreme Court's decision in Brown v. Board of Education (1954), which declared state laws establishing separate public schools for Black and white students to be unconstitutional.
The 1950s and 1960s are often regarded as the height of the civil rights movement, a period characterized by widespread activism, landmark legislation, and profound social change. The movement was marked by nonviolent protests, such as the Montgomery Bus Boycott (1955-1956) and the March on Washington (1963), where Dr. Martin Luther King Jr. delivered his iconic "I Have a Dream" speech. These actions, along with the courage and determination of countless individuals, brought national attention to the injustices faced by African Americans.
Key legislative achievements during this period included the Civil Rights Act of 1964 and the Voting Rights Act of 1965. The Civil Rights Act prohibited discrimination on the basis of race, color, religion, sex, or national origin and ended segregation in public places. The Voting Rights Act aimed to eliminate barriers to voting for African Americans, particularly in the South, where practices such as literacy tests and poll taxes had been used to disenfranchise Black voters.
Despite these significant advancements, the struggle for civil rights continued beyond the 1960s. The late 20th and early 21st centuries saw ongoing efforts to address issues such as racial profiling, police brutality, and economic inequality. The Black Lives Matter movement, which emerged in response to the killings of unarmed Black individuals by police, has brought renewed attention to the persistent racial disparities in American society.
The quest for civil rights in the United States has not been limited to African Americans. Other marginalized groups, including women, Native Americans, LGBTQ+ individuals, and immigrants, have also fought for recognition and equality. The women's suffrage movement, which culminated in the 19th Amendment in 1920, granted women the right to vote and marked a significant step toward gender equality. The LGBTQ+ rights movement achieved a major victory with the Supreme Court's decision in Obergefell v. Hodges (2015), which legalized same-sex marriage nationwide.
Throughout American history, the struggle for civil rights has been a dynamic and multifaceted endeavor. It has involved legal battles, grassroots activism, and significant sacrifices by individuals committed to the cause of equality. The progress achieved has been substantial, yet the journey is ongoing, with new challenges and opportunities for advancing civil rights continuing to emerge.
In conclusion, the history of civil rights in the United States is a testament to the resilience and determination of those who have fought for justice and equality. From the abolition of slavery to the civil rights movement of the 1960s and beyond, each generation has contributed to the ongoing effort to realize the ideals of freedom and equality upon which the nation was founded. As society continues to evolve, the legacy of these struggles serves as both a reminder of the progress made and a call to action to address the inequalities that persist today. The journey toward equality is far from over, but the commitment to civil rights remains a defining aspect of the American spirit. Each new movement and each legal victory brings the nation closer to the ideal of true equality, demonstrating that the pursuit of civil rights is an enduring and essential part of the American story. The impact of these movements reverberates through history, showcasing the power of collective action and the relentless pursuit of justice. As new challenges arise, the foundational values of equality and justice continue to guide and inspire future generations to push forward.
"""


In [11]:
# Generate grading feedback
grading_feedback = analyze_essay_with_rubric(model, rubric_content, essay_text, additional_info)
print("Initial Grading Feedback:")
print(grading_feedback)
display(to_markdown(grading_feedback))


Rubric Token Count: 359
Essay Token Count: 1270
Additional Info Token Count: 5
Grading Prompt Token Count: 1881
Grading Response Token Count: 720
Grading Total Token Count: 2601
 
Initial Grading Feedback:
### Grading Rubric and Feedback

No. The provided rubric primarily focuses on scientific inquiry skills and is not appropriate for grading an essay on the history of the civil rights movement in the United States. 

Here's a more appropriate rubric for assessing the essay:

##  U.S. Civil Rights Movement Essay Rubric 

| Criterion                                          | 1 (Needs Improvement)                                        | 2 (Developing)                                       | 3 (Proficient)                                        | 4 (Excellent)                                       | 5 (Exemplary)                                        |
|----------------------------------------------------|--------------------------------------------------------------|------------------

> ### Grading Rubric and Feedback
> 
> No. The provided rubric primarily focuses on scientific inquiry skills and is not appropriate for grading an essay on the history of the civil rights movement in the United States. 
> 
> Here's a more appropriate rubric for assessing the essay:
> 
> ##  U.S. Civil Rights Movement Essay Rubric 
> 
> | Criterion                                          | 1 (Needs Improvement)                                        | 2 (Developing)                                       | 3 (Proficient)                                        | 4 (Excellent)                                       | 5 (Exemplary)                                        |
> |----------------------------------------------------|--------------------------------------------------------------|-------------------------------------------------------|-------------------------------------------------------------|------------------------------------------------------|-------------------------------------------------------------------|
> | **Content (30%)**                                  | Information is inaccurate, incomplete, or irrelevant.  | Content is superficial or contains some inaccuracies. | Content is generally accurate and relevant to the topic. | Content is accurate, relevant, and shows depth of understanding. | Content demonstrates insightful analysis and a nuanced understanding.| 
> | **Analysis & Critical Thinking (25%)**            | Lacks analysis or critical engagement with the topic.       | Analysis is simplistic or lacks sufficient support.   | Provides some analysis and interpretation of events and ideas.| Offers insightful analysis and connects events to broader themes.| Demonstrates sophisticated analysis and original insights.        |
> | **Structure & Organization (20%)**               | Essay lacks clear structure and organization.                | Organization is somewhat unclear; transitions are weak.  | Essay is organized with a clear introduction, body, and conclusion. | Structure enhances the clarity and flow of ideas.     | Structure is logical, engaging, and effectively highlights key points.|
> | **Evidence & Examples (15%)**                    | Evidence is lacking or irrelevant.                           | Evidence is weak or not always relevant.              | Provides relevant evidence to support claims.            | Evidence is well-chosen and effectively supports the analysis. | Evidence is meticulously chosen and skillfully used to build a compelling case. |
> | **Clarity & Mechanics (10%)**                    | Writing is unclear, contains frequent errors, or is difficult to understand. | Writing contains some errors that distract the reader. | Writing is generally clear and error-free.            | Writing is clear, concise, and error-free.            | Writing is elegant, engaging, and demonstrates mastery of language. |
> 
> 
> ## Overall Feedback
> 
> While the essay demonstrates a good understanding of the chronology and key events of the civil rights movement, it relies heavily on summarization and lacks in-depth analysis and critical thinking. To improve, consider the following:
> 
> * **Go Beyond Summarizing:** Instead of just describing events, delve deeper into their causes, consequences, and significance. Analyze the motivations and strategies of different groups and individuals. 
> * **Develop a Clear Argument:** What is the main point you want to make about the history of the civil rights movement? Having a central argument will help you focus your analysis and make your essay more engaging.
> * **Provide Specific Examples:**  Instead of making general statements, support your points with specific examples from history. This will make your essay more persuasive and demonstrate a deeper understanding of the topic.
> * **Engage with Different Perspectives:**  Consider different viewpoints and interpretations of historical events. This will add nuance and complexity to your analysis.
> * **Proofread Carefully:**  While there are no major grammatical errors, carefully proofreading your work will help eliminate minor mistakes and improve the overall clarity of your writing. 


In [12]:
# Validate the feedback and re-grade if necessary
validation_feedback = validate_feedback(validation_model, grading_feedback, rubric_content, essay_text, additional_info)
print("Validation Feedback:")
print(validation_feedback)

# Check if the feedback was re-graded
corrected_feedback_match = re.search(r'Corrected Grading Feedback:\n(.*)', validation_feedback, re.DOTALL)
if corrected_feedback_match:
    grading_feedback = corrected_feedback_match.group(1).strip()
    print("Corrected Grading Feedback:")
    display(to_markdown(grading_feedback))
else:
    print("The original grading feedback was valid.")
    display(to_markdown(grading_feedback))


Validation Response Token Count: 476
Validation Total Token Count: 3048
 
Validation Feedback:
The provided grading feedback is not valid because it disregards the given instructions by dismissing the provided rubric and creating a new one. 

Here is a corrected grading feedback following the given rubric and instructions:

##  Grading Rubric and Feedback

**# New York State P-12 Science Learning Standards Rubric**

| Criterion                                          | 1                                        | 2                                        | 3                                        | 4                                        | 5                                        |
|----------------------------------------------------|------------------------------------------|------------------------------------------|------------------------------------------|------------------------------------------|------------------------------------------|
| **1. Developing and Using Models (20%)

> ### Grading Rubric and Feedback
> 
> No. The provided rubric primarily focuses on scientific inquiry skills and is not appropriate for grading an essay on the history of the civil rights movement in the United States. 
> 
> Here's a more appropriate rubric for assessing the essay:
> 
> ##  U.S. Civil Rights Movement Essay Rubric 
> 
> | Criterion                                          | 1 (Needs Improvement)                                        | 2 (Developing)                                       | 3 (Proficient)                                        | 4 (Excellent)                                       | 5 (Exemplary)                                        |
> |----------------------------------------------------|--------------------------------------------------------------|-------------------------------------------------------|-------------------------------------------------------------|------------------------------------------------------|-------------------------------------------------------------------|
> | **Content (30%)**                                  | Information is inaccurate, incomplete, or irrelevant.  | Content is superficial or contains some inaccuracies. | Content is generally accurate and relevant to the topic. | Content is accurate, relevant, and shows depth of understanding. | Content demonstrates insightful analysis and a nuanced understanding.| 
> | **Analysis & Critical Thinking (25%)**            | Lacks analysis or critical engagement with the topic.       | Analysis is simplistic or lacks sufficient support.   | Provides some analysis and interpretation of events and ideas.| Offers insightful analysis and connects events to broader themes.| Demonstrates sophisticated analysis and original insights.        |
> | **Structure & Organization (20%)**               | Essay lacks clear structure and organization.                | Organization is somewhat unclear; transitions are weak.  | Essay is organized with a clear introduction, body, and conclusion. | Structure enhances the clarity and flow of ideas.     | Structure is logical, engaging, and effectively highlights key points.|
> | **Evidence & Examples (15%)**                    | Evidence is lacking or irrelevant.                           | Evidence is weak or not always relevant.              | Provides relevant evidence to support claims.            | Evidence is well-chosen and effectively supports the analysis. | Evidence is meticulously chosen and skillfully used to build a compelling case. |
> | **Clarity & Mechanics (10%)**                    | Writing is unclear, contains frequent errors, or is difficult to understand. | Writing contains some errors that distract the reader. | Writing is generally clear and error-free.            | Writing is clear, concise, and error-free.            | Writing is elegant, engaging, and demonstrates mastery of language. |
> 
> 
> ## Overall Feedback
> 
> While the essay demonstrates a good understanding of the chronology and key events of the civil rights movement, it relies heavily on summarization and lacks in-depth analysis and critical thinking. To improve, consider the following:
> 
> * **Go Beyond Summarizing:** Instead of just describing events, delve deeper into their causes, consequences, and significance. Analyze the motivations and strategies of different groups and individuals. 
> * **Develop a Clear Argument:** What is the main point you want to make about the history of the civil rights movement? Having a central argument will help you focus your analysis and make your essay more engaging.
> * **Provide Specific Examples:**  Instead of making general statements, support your points with specific examples from history. This will make your essay more persuasive and demonstrate a deeper understanding of the topic.
> * **Engage with Different Perspectives:**  Consider different viewpoints and interpretations of historical events. This will add nuance and complexity to your analysis.
> * **Proofread Carefully:**  While there are no major grammatical errors, carefully proofreading your work will help eliminate minor mistakes and improve the overall clarity of your writing. 


In [13]:
# Generate suggestions and rewrite the essay based on the validated or corrected feedback
suggestions_and_rewrite = generate_suggestions_and_rewrite_essay(model, grading_feedback, essay_text)
print(suggestions_and_rewrite)


Grading Feedback Token Count: 722
Essay Token Count: 1270
Suggestions Prompt Token Count: 2246
Suggestions Response Token Count: 1053
Suggestions Total Token Count: 3299
 
Full response from model:
 [
  {
    "improvement": "Go Beyond Summarizing: Analyze the motivations and strategies of different groups and individuals.",
    "criterion_from_rubric": "Analysis & Critical Thinking",
    "reason_for_suggestion": "The essay primarily summarizes historical events without delving into the complexities of motivations, strategies, and perspectives.",
    "original_text": "The abolitionist movement of the 19th century was one of the earliest organized efforts to challenge slavery and advocate for the civil rights of African Americans. Figures such as Frederick Douglass, Harriet Tubman, and William Lloyd Garrison played pivotal roles in raising awareness and mobilizing support for the abolition of slavery.",
    "revised_text": "The abolitionist movement of the 19th century employed diverse s

In [14]:
extracted_data = extract_json_improvements(suggestions_and_rewrite)
print("Extracted Data:", extracted_data)



Cleaned JSON string: [
  {
    "improvement": "Go Beyond Summarizing: Analyze the motivations and strategies of different groups and individuals.",
    "criterion_from_rubric": "Analysis & Critical Thinking",
    "reason_for_suggestion": "The essay primarily summarizes historical events without delving into the complexities of motivations, strategies, and perspectives.",
    "original_text": "The abolitionist movement of the 19th century was one of the earliest organized efforts to challenge slavery and advocate for the civil rights of African Americans. Figures such as Frederick Douglass, Harriet Tubman, and William Lloyd Garrison played pivotal roles in raising awareness and mobilizing support for the abolition of slavery.",
    "revised_text": "The abolitionist movement of the 19th century employed diverse strategies to dismantle slavery.  Figures like Frederick Douglass, a powerful orator and writer, exposed the horrors of slavery firsthand.  Harriet Tubman, a conductor on the Unde

In [15]:
display(Markdown(f"**Suggestions and Revised Essay:**\n\n{extracted_data['response_without_json']}"))


**Suggestions and Revised Essay:**

 


In [16]:
# Display the improvements in the new essay with hover effect
display_improvements(extracted_data['improvements'], essay_text)
